In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
url = "https://forecast.weather.gov/MapClick.php?lat=37.77493000000004&lon=-122.41941999999995"

In [3]:
content = requests.get(url).content
# get soup
soup = BeautifulSoup(content,'html.parser') # lxml parser

In [4]:
weather_table = soup.find_all('div',class_="panel-body")[2] #{id:"detailed-forecast-body"})

In [5]:
weather_data_rows = weather_table.find_all("div", class_="row-forecast")

In [6]:
len(weather_data_rows)

14

In [7]:
day_nights = []
descriptions = []
temperatures = []

for each in weather_data_rows:
    col_data = each.find_all("div")
    day_night = col_data[0].text
    description = col_data[1].text
    day_nights.append(day_night)
    descriptions.append(description)
#     print(day_night,":", description)
#     print()

In [8]:
temperatures = []
for each in descriptions:
    temperature = each.split(".")[0][-2:]
#     print(temperature)
    temperatures.append(temperature)

In [9]:
short_descs = []
for each in descriptions:
    short_desc = each.split(",")[0]
    short_descs.append(short_desc)
#     print(short_desc)

In [10]:
sf_time = datetime.today() - timedelta(hours=12, minutes=30)
periods = []
for each in range(len(day_nights)):
    period = (sf_time + timedelta(hours = each*12)).strftime("%a %d, %b")
    periods.append(period)
#     print(period)

In [11]:
data_dict = {
    "Period" : periods,
    "Short Desc" : short_descs,
    "Temperature" : temperatures,
    "Description" : descriptions
}

In [12]:
df = pd.DataFrame(data_dict)
df.index = df.index+1
df['Day Time'] = "Day"

In [13]:
for index in range(len(df)):
    if (index%2!=0):
        df.at[index,"Day Time"] = "Night"

In [14]:
df.head(20)

,Period,Short Desc,Temperature,Description,Day Time
1,"Wed 25, Aug",Mostly cloudy,54,"Mostly cloudy, with a low around 54. West wind...",Night
2,"Wed 25, Aug",Cloudy through mid morning,66,"Cloudy through mid morning, then gradual clear...",Day
3,"Thu 26, Aug",Partly cloudy,55,"Partly cloudy, with a low around 55. Breezy, w...",Night
4,"Thu 26, Aug",Sunny,72,"Sunny, with a high near 72. Light south southe...",Day
5,"Fri 27, Aug",Mostly clear,57,"Mostly clear, with a low around 57. Southwest ...",Night
6,"Fri 27, Aug",Sunny,74,"Sunny, with a high near 74.",Day
7,"Sat 28, Aug",Mostly clear,57,"Mostly clear, with a low around 57.",Night
8,"Sat 28, Aug",Sunny,76,"Sunny, with a high near 76.",Day
9,"Sun 29, Aug",Mostly clear,58,"Mostly clear, with a low around 58.",Night
10,"Sun 29, Aug",Sunny,72,"Sunny, with a high near 72.",Day
